## Install packages

In [ ]:
#pip install -r requirements.txt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#  !git clone --single-branch --branch fast_tokenizers_BARTpho_PhoBERT_BERTweet https://github.com/datquocnguyen/transformers.git

Cloning into 'transformers'...
remote: Enumerating objects: 138580, done.
remote: Counting objects: 100% (47/47), done.
remote: Compressing objects: 100% (46/46), done.
remote: Total 138580 (delta 1), reused 47 (delta 1), pack-reused 138533
Receiving objects: 100% (138580/138580), 145.03 MiB | 20.34 MiB/s, done.
Resolving deltas: 100% (104129/104129), done.


In [ ]:
!pip install -e transformers/

Obtaining file:///content/transformers
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.1/311.1 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 47.9 MB/s eta 0:00:00
  Building editable for transformers (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.32.0.dev0-0.editable-py3-none-any.whl size=38187 sha256=d3d690e8f56c467ca9d942ec81c3189773fd28027ffce03161a2b6d0a84540bf
  Stored in directory: /tmp/pip-ephem-wheel-cache-11579rmb/wheels/7c/35/80/e946b22a081210c6642e607ed65b2a5b9a4d9259695ee2caf5
Successfully built transformers


In [ ]:
# !pip install datasets
# !pip install evaluate
# !pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
# !pip install transformers[torch]
# !pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 3.6 MB/s eta 0:00:00


In [ ]:
# MODEL = "vinai/bartpho-word-base"
MODEL = "viettel-phobert-finetune-viquad"
DATASET_FOLDER = 'drive/MyDrive/DACN/NoteBook02/dataset02'

## Create data

In [ ]:
import os
import json

def save_data(data, type):
    # save your preprocessed data
    with open(os.path.join(DATASET_FOLDER + "/merge_dataset/", type + ".json"), "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent= 4)
    return

In [ ]:
import torch
import json
from transformers import BartphoTokenizerFast
#import transformers

In [ ]:
from datasets import Dataset, DatasetDict

## Load data & Train model

# New Section

In [ ]:
def readAllData(folder):
  jsonFiles = []
  for fileName in os.listdir(folder):
    if fileName.endswith('.json'):
      filePath = os.path.join(folder, fileName)
      with open(filePath, 'rb') as f:
        fileData = json.load(f)
        jsonFiles.append(fileData)

  data = {
      'train': [],
      'validation': [],
      'test': []
  }
  for f in jsonFiles:
    data['train'] += (f['train']['data'])
    data['validation'] += f['validation']['data']
    data['test'] = []

  return data

In [ ]:
all_data = readAllData(DATASET_FOLDER)

In [ ]:
save_data(all_data['train'], "train_data")
save_data(all_data['validation'], "validation_data")
save_data(all_data['test'], "test_data")

In [ ]:
train_data = Dataset.from_json(DATASET_FOLDER + "/merge_dataset/train_data.json")
validation_data = Dataset.from_json(DATASET_FOLDER + "/merge_dataset/validation_data.json")
test_data = Dataset.from_json(DATASET_FOLDER + "/merge_dataset/test_data.json")

raw_datasets = DatasetDict({'train': train_data, 'validation': validation_data, 'test': test_data})

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
print(raw_datasets)

DatasetDict({
    train: Dataset({
        features: ['answers', 'title', 'question', 'context', 'id'],
        num_rows: 747
    })
    validation: Dataset({
        features: ['answers', 'title', 'question', 'context', 'id'],
        num_rows: 23
    })
    test: Dataset({
        features: [],
        num_rows: 0
    })
})


In [ ]:
print("Context: ", raw_datasets["train"][0]["context"])
print("Question: ", raw_datasets["train"][0]["question"])
print("Answer: ", raw_datasets["train"][0]["answers"])

Context:  1. Quy định: SV xem quy định tín chỉ TCTD tại trang hcmut.edu.vn >> Đào tạo >> Chương trình đào tạo >> Quyết định ban hành quy định về cấu trúc và quy trình xây dựng, thẩm định hiệu chỉnh chương trình đào tạo/ Điều 2 mục 6: "Là tín chỉ của các học phần người học được tự do đăng ký, không bị ràng buộc được xem xét khi tích luỹ các học phần có kiến thức không trùng lắp với các kiến thức dùng để xét tốt nghiệp của chương trình đào tạo và không phải là các học phần thuộc khối kiến thức tốt nghiệp của một ngành khác. Hai học phần được xem là trùng lắp nếu có chuẩn đầu ra/nội dung trùng lắp đến 50%". 2. Nguyên tắc xét: - Đại cương (Toán, Khoa học tự nhiên, M -Lenin, Pháp luật Việt Nam đại cương, Các môn ngoại ngữ ...): không là tự chọn tự do - Nhập môn: không là tự chọn tự do - Các môn học trong danh sách loại trừ: không là tự chọn tự do - Các môn học trong cùng nhóm trùng lắp, tương đương: chỉ tính 01 lần (Ví dụ: 2 môn CO1009 4TC và CO1023 3TC cùng là Hệ thống số, chỉ được tính 01

During training, there is only **ONE** possible answer. We have to filter datasets which has only **ONE** possible answer.

In [ ]:
raw_datasets["train"] = raw_datasets["train"].filter(lambda x: len(x["answers"]["text"]) == 1)
raw_datasets["train"] = raw_datasets["train"].filter(lambda x: len(x["answers"]["answer_start"]) == 1)

Filter:   0%|          | 0/747 [00:00<?, ? examples/s]

Filter:   0%|          | 0/741 [00:00<?, ? examples/s]

In [ ]:
raw_datasets["validation"] = raw_datasets["validation"].filter(lambda x: len(x["answers"]["text"]) == 1)
raw_datasets["validation"] = raw_datasets["validation"].filter(lambda x: len(x["answers"]["answer_start"]) == 1)

Filter:   0%|          | 0/23 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1 [00:00<?, ? examples/s]

In [ ]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['answers', 'title', 'question', 'context', 'id'],
        num_rows: 730
    })
    validation: Dataset({
        features: ['answers', 'title', 'question', 'context', 'id'],
        num_rows: 1
    })
    test: Dataset({
        features: [],
        num_rows: 0
    })
})

git clone --single-branch --branch fast_tokenizers_BARTpho_PhoBERT_BERTweet https://github.com/datquocnguyen/transformers.git

cd transformers

pip3 install -e .

In [ ]:
raw_datasets['validation'][0]

{'answers': {'answer_end': [2322],
  'answer_start': [2262],
  'text': ['sinh viên không có lịch học trong thời gian hè được nghỉ hè.']},
 'title': 'Quy chế',
 'question': 'Sinh viên có lịch học hè không?',
 'context': 'Chương 3. KẾ HOẠCH GIẢNG DẠY Điều 6. Tổ chức năm học 6.1 Tổ chức các học kỳ Trường tổ chức hai học kỳ chính trong một năm học. Mỗi học kỳ có 15 tuần học và từ 2 đến 3 tuần đánh giá học phần. Ngoài ra, trường có thể tổ chức thêm một học kỳ phụ trong thời gian hè, trong khoảng từ 6 đến 13 tuần, bao gồm thời gian đánh giá học phần. Ký hiệu các học kỳ có dạng HKYYS, trong đó YY là hai số cuối của năm và S là số thứ tự học kỳ. Ví dụ: HK211, HK212 và HK213 là các học kỳ 1, học kỳ 2 và học kỳ hè của năm học 2021-2022. 6.2 Biểu đồ học tập và lịch học vụ Biểu đồ kế hoạch học tập và giảng dạy (gọi tắt là Biểu đồ học tập) là một kế hoạch có tính bắt buộc thực hiện trong toàn trường. Biểu đồ quy định các hoạt động học tập và giảng dạy và khung thời gian cho từng hoạt động. Biểu đồ 

In [ ]:
tokenizer = BartphoTokenizerFast.from_pretrained(MODEL)
tokenizer.is_fast

(…)ho-word-base/resolve/main/tokenizer.json:   0%|          | 0.00/3.13M [00:00<?, ?B/s]

(…)rtpho-word-base/resolve/main/config.json:   0%|          | 0.00/898 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'PhobertTokenizer'. 
The class this function is called from is 'BartphoTokenizerFast'.


True

or you can use:
```(python)
import torch
from transformers import PreTrainedTokenizerFast

tokenizer = PreTrainedTokenizerFast.from_pretrained("vinai/bartpho-syllable")
tokenizer.is_fast
```

In [ ]:
# context = raw_datasets["train"][0]["context"]
# question = raw_datasets["train"][0]["question"]

# inputs = tokenizer(question, context)
# tokenizer.decode(inputs["input_ids"])

In [ ]:
# inputs = tokenizer(
#     question,
#     context,
#     max_length=100,
#     truncation="only_second",
#     stride=50,
#     return_overflowing_tokens=True,
# )

# for ids in inputs["input_ids"]:
#     print(tokenizer.decode(ids))
#     print()

In [ ]:
# from datasets import load_dataset

# cuant = load_dataset('squad_v2')

In [ ]:
# print(cuant['train'][66939]['answers']['text'])
# print(cuant['train'][66939]['context'])
# print(cuant['train'][443]['context'])
# max = 0
# for i in range(len(cuant['train'])):
  # if len(cuant['train'][i]['answers']['text']) == 1:
  #   if len(cuant['train'][i]['answers']['text'][0]) > max:
  #     max = i
#   if len(cuant['train'][i]['answers']['text']) == 0:
#     max = i
#     break
# print(max)

# print(cuant['train'][2075]['answers'])

In [ ]:
# def identify_answer_start(input_ids, answer, question, tokenizer = tokenizer):
#   result = []
#   for ids in input_ids:
#     short_context = tokenizer.decode(ids)[len(question) + 12 : len(tokenizer.decode(ids))  - 4]
#     found_idx = short_context.find(answer)
#     result.append([found_idx] if found_idx >= 0 else [0])
#   return result

In [ ]:
# answer = raw_datasets['train'][0]['answers']['text'][0][:50]
# print(answer)
# for i, index in inputs['input_ids']:
#   short_context = tokenizer.decode(i)
#   if index == 0:
#     print()
#   res = short_context.find(answer)
#   print(res)
#   print()


In [ ]:
# question = raw_datasets['train'][4]['question']
# answers = raw_datasets['train'][4]['answers']
# context = raw_datasets['train'][4]['context']

# print(context)
# print()
# print(question)
# print()
# print(answers)
# print()

# inputs = tokenizer(
#     question,
#     context,
#     max_length=100,
#     truncation="only_second",
#     stride=50,
#     return_overflowing_tokens=True,
# )

# for ids in inputs["input_ids"]:
#     print(tokenizer.decode(ids)[len(question) + 12 : len(tokenizer.decode(ids))  - 4])
#     print(tokenizer.decode(ids))
#     print()

`max_length` to set the maximum length (here 100)

`truncation="only_second"` to truncate the context (which is in the second position) when the question with its context is too long

`stride` to set the number of overlapping tokens between two successive chunks (here 50)

`return_overflowing_tokens=True` to let the tokenizer know we want the overflowing tokens

This is how `stride` works:

In [ ]:
# inputs = tokenizer(
#     question,
#     context,
#     max_length=100,
#     truncation="only_second",
#     stride=50,
#     return_overflowing_tokens=True,
#     return_offsets_mapping=True,
# )
# inputs.keys()

In [ ]:
# inputs["overflow_to_sample_mapping"]

In [ ]:
# inputs = tokenizer(
#     raw_datasets["train"][2:5]["question"],
#     raw_datasets["train"][2:5]["context"],
#     max_length=100,
#     truncation="only_second",
#     stride=50,
#     return_overflowing_tokens=True,
#     return_offsets_mapping=True,
# )

# print(f"The 3 examples gave {len(inputs['input_ids'])} features.")
# print(f"Here is where each comes from: {inputs['overflow_to_sample_mapping']}.")

In [ ]:
# raw_datasets["train"][2:5]["answers"]

In [ ]:
# inputs

In [ ]:
# for i, offset in enumerate(inputs["offset_mapping"]):
    # print(i, offset)

`sequence_ids` similar to Next Sentence Prediction

In [ ]:
# inputs.sequence_ids(0)

In [ ]:
# inputs["overflow_to_sample_mapping"][0]

In [ ]:
# answers = raw_datasets["train"][2:5]["answers"]
# start_positions = []
# end_positions = []

# for i, offset in enumerate(inputs["offset_mapping"]):
#     sample_idx = inputs["overflow_to_sample_mapping"][i]
#     answer = answers[sample_idx]
#     start_char = answer["answer_start"][0]
#     end_char = answer["answer_start"][0] + len(answer["text"][0])
#     sequence_ids = inputs.sequence_ids(i)

#     # Find the start and end of the context
#     idx = 0
#     while sequence_ids[idx] != 1:
#         idx += 1
#     context_start = idx
#     while sequence_ids[idx] == 1:
#         idx += 1
#     context_end = idx - 1

#     # If the answer is not fully inside the context, label is (0, 0)
#     if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
#         start_positions.append(0)
#         end_positions.append(0)
#     else:
#         # Otherwise it's the start and end token positions
#         idx = context_start
#         while idx <= context_end and offset[idx][0] <= start_char:
#             idx += 1
#         start_positions.append(idx - 1)

#         idx = context_end
#         while idx >= context_start and offset[idx][1] >= end_char:
#             idx -= 1
#         end_positions.append(idx + 1)

# start_positions, end_positions

In [ ]:
# idx = 0
# sample_idx = inputs["overflow_to_sample_mapping"][idx]
# answer = answers[sample_idx]["text"][0]

# start = start_positions[idx]
# end = end_positions[idx]
# labeled_answer = tokenizer.decode(inputs["input_ids"][idx][start : end + 1])

# print(f"Theoretical answer: {answer}, labels give: {labeled_answer}")

Now let’s check index 4, where we set the labels to (0, 0), which means the answer is not in the context chunk of that feature

In [ ]:
# idx = 4
# sample_idx = inputs["overflow_to_sample_mapping"][idx]
# answer = answers[sample_idx]["text"][0]

# decoded_example = tokenizer.decode(inputs["input_ids"][idx])
# print(f"Theoretical answer: {answer}, decoded example: {decoded_example}")

Indeed, we don’t see the answer inside the context.

In [ ]:
# max_length = 1024
max_length = 1024
stride = 256

def preprocess_training_dataset(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    sample_map = inputs.pop("overflow_to_sample_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        sample_idx = sample_map[i]
        answer = answers[sample_idx]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label is (0, 0)
        if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [ ]:
train_dataset = raw_datasets["train"].map(
    preprocess_training_dataset,
    batched=True,
    remove_columns=raw_datasets["train"].column_names,
)
len(raw_datasets["train"]), len(train_dataset)

Map:   0%|          | 0/730 [00:00<?, ? examples/s]

(730, 1683)

In [ ]:
# inputs.pop("overflow_to_sample_mapping")

A tiny bit of cleanup of the **offset mappings**. They will contain offsets for the *question* and the *context*, but once we’re in the post-processing stage we won’t have any way to know which part of the input IDs corresponded to the *context* and which part was the *question*. So, we’ll set the offsets corresponding to the *question* to `None`:

In [ ]:
# max_length = 500
# stride = 250

In [ ]:
def preprocess_validation_dataset(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_map = inputs.pop("overflow_to_sample_mapping")
    example_ids = []

    for i in range(len(inputs["input_ids"])):
        sample_idx = sample_map[i]
        example_ids.append(examples["id"][sample_idx])

        sequence_ids = inputs.sequence_ids(i)
        offset = inputs["offset_mapping"][i]
        inputs["offset_mapping"][i] = [
            o if sequence_ids[k] == 1 else None for k, o in enumerate(offset)
        ]

    inputs["example_id"] = example_ids
    return inputs

In [ ]:
validation_dataset = raw_datasets["validation"].map(
    preprocess_validation_dataset,
    batched=True,
    remove_columns=raw_datasets["validation"].column_names,
)
len(raw_datasets["validation"]), len(validation_dataset)

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

(1, 2)

In [ ]:
validation_dataset

Dataset({
    features: ['input_ids', 'attention_mask', 'offset_mapping', 'example_id'],
    num_rows: 2
})

In [ ]:
from tqdm.auto import tqdm
import evaluate
import numpy as np
import collections

n_best = 20
max_answer_length = 30
predicted_answers = []
metric = evaluate.load("squad")

def compute_metrics(start_logits, end_logits, features, examples):
    example_to_features = collections.defaultdict(list)
    for idx, feature in enumerate(features):
        example_to_features[feature["example_id"]].append(idx)

    predicted_answers = []
    for example in tqdm(examples):
        example_id = example["id"]
        context = example["context"]
        answers = []

        # Loop through all features associated with that example
        for feature_index in example_to_features[example_id]:
            start_logit = start_logits[feature_index]
            end_logit = end_logits[feature_index]
            offsets = features[feature_index]["offset_mapping"]

            start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
            end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Skip answers that are not fully in the context
                    if offsets[start_index] is None or offsets[end_index] is None:
                        continue
                    # Skip answers with a length that is either < 0 or > max_answer_length
                    if (
                        end_index < start_index
                        or end_index - start_index + 1 > max_answer_length
                    ):
                        continue

                    answer = {
                        "text": context[offsets[start_index][0] : offsets[end_index][1]],
                        "logit_score": start_logit[start_index] + end_logit[end_index],
                    }
                    answers.append(answer)

        # Select the answer with the best score
        if len(answers) > 0:
            best_answer = max(answers, key=lambda x: x["logit_score"])
            predicted_answers.append(
                {"id": example_id, "prediction_text": best_answer["text"]}
            )
        else:
            predicted_answers.append({"id": example_id, "prediction_text": ""})

    theoretical_answers = [{"id": ex["id"], "answers": ex["answers"]} for ex in examples]
    return metric.compute(predictions=predicted_answers, references=theoretical_answers)

In [ ]:
#!pip install transformers[torch]

In [ ]:
#!pip install accelerate -U

In [ ]:
# Check on the available device - use GPU
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f'Working on {device}')

Working on cuda


In [ ]:
# !pip install transformers[torch]
# !pip install accelerate -U

In [ ]:
from transformers import Trainer, TrainingArguments
from transformers import AutoModelForQuestionAnswering

# model = AutoModelForQuestionAnswering.from_pretrained("vinai/bartpho-word-base")
model = AutoModelForQuestionAnswering.from_pretrained(MODEL)

epochs = 5
batch_size = 4
lr = 2e-5

training_args = TrainingArguments(
    output_dir = "checkpoints",
    evaluation_strategy = "steps", #print evaluation after finishing an epoch
    num_train_epochs=epochs,
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    save_total_limit=1,
    save_steps=2000,
    eval_steps=2000,
    gradient_accumulation_steps=2,
    eval_accumulation_steps=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    compute_metrics=compute_metrics
)
trainer.train()

pytorch_model.bin:   0%|          | 0.00/600M [00:00<?, ?B/s]

Some weights of MBartForQuestionAnswering were not initialized from the model checkpoint at vinai/bartpho-word-base and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/content/transformers/src/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss


TrainOutput(global_step=1050, training_loss=1.9866852242606028, metrics={'train_runtime': 2323.254, 'train_samples_per_second': 3.622, 'train_steps_per_second': 0.452, 'total_flos': 5119585110343680.0, 'train_loss': 1.9866852242606028, 'epoch': 4.99})

In [ ]:
# Chỗ này lưu Modal train => Đừng có chạy tầm bậy
# model_path = '/content/drive/MyDrive/DACN/ChatBotTrainModel03'
# model.save_pretrained(model_path)
# tokenizer.save_pretrained(model_path)

('/content/drive/MyDrive/DACN/ChatBotTrainModel03/tokenizer_config.json',
 '/content/drive/MyDrive/DACN/ChatBotTrainModel03/special_tokens_map.json',
 '/content/drive/MyDrive/DACN/ChatBotTrainModel03/tokenizer.json')

In [ ]:
model_path = '/content/drive/MyDrive/DACN/ChatBotTrainModel03'

In [ ]:
import transformers
import torch

In [ ]:
from transformers import AutoModelForQuestionAnswering, BartphoTokenizerFast

In [ ]:
# !pip install datasets

  Using cached datasets-2.14.6-py3-none-any.whl (493 kB)
  Using cached dill-0.3.7-py3-none-any.whl (115 kB)
  Using cached multiprocess-0.70.15-py310-none-any.whl (134 kB)


In [ ]:

model = AutoModelForQuestionAnswering.from_pretrained(model_path)
tokenizer = BartphoTokenizerFast.from_pretrained(model_path)



device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f'Working on {device}')
model = model.to(device)

Working on cuda


In [ ]:
tokenizer.is_fast

True

In [ ]:
# !pip install uuid

  Preparing metadata (setup.py) ... done
  Created wheel for uuid: filename=uuid-1.30-py3-none-any.whl size=6479 sha256=00fa75b2d5174a79b4447b8d7617e817bc538ffd2940eb26fd445f8fc307ff79
  Stored in directory: /root/.cache/pip/wheels/ed/08/9e/f0a977dfe55051a07e21af89200125d65f1efa60cbac61ed88
Successfully built uuid


In [ ]:
import uuid

def remove_dup_elements(dataset, column_name = 'context'):
  column_data = dataset[column_name]
  column_set = set(column_data)
  unique_elements = list(column_set)
  return unique_elements

def get_all_contexts(raw_datasets = raw_datasets):
  train_contexts = raw_datasets['train']
  unique_contexts = remove_dup_elements(train_contexts, 'context')
  context_length = len(unique_contexts)
  context_ids = []
  for i in unique_contexts:
    new_id = uuid.uuid4()
    context_ids.append(str(new_id))

  #return train_unique_contexts.remove_columns(['answers', 'question'])
  return Dataset.from_dict({'context': unique_contexts, 'id': context_ids})


In [ ]:
all_contexts = get_all_contexts(raw_datasets)


In [ ]:
print(all_contexts)

Dataset({
    features: ['context', 'id'],
    num_rows: 42
})


In [ ]:
# def get_prediction(question, contexts, n_best = 20):
#   new_column = [question] * len(contexts)
#   eval_long_set = contexts.add_column("question", new_column)
#   # eval_long_set = contexts.map(lambda context: {"context": context, "question": question})
#   print(eval_long_set[1])
#   eval_set = eval_long_set.map(
#     preprocess_validation_dataset,
#     batched=True,
#     remove_columns=raw_datasets["test"].column_names,
#     load_from_cache_file=False
#   )
#   eval_set_for_model = eval_set.remove_columns(["example_id", "offset_mapping"])
#   eval_set_for_model.set_format("torch")


#   batch = {k: eval_set_for_model[k].to("cuda") for k in eval_set_for_model.column_names}

#   outputs = model(**batch)
#   print(outputs)
#   start_logits = outputs.start_logits.detach().cpu().numpy()
#   end_logits = outputs.end_logits.detach().cpu().numpy()

#   for index in range(len(start_logits)):
#     start_logit = start_logits[index]
#     end_logit = end_logits[index]
#     offsets = eval_set["offset_mapping"][index]

#     start_indexes = np.argsort(start_logit)[-1 : - n_best - 1 : -1].tolist()
#     end_indexes = np.argsort(end_logit)[-1 : - n_best - 1 : -1].tolist()

#     for start_index in start_indexes:
#       for end_index in end_indexes:
#         # Skip answers that are not fully in the context
#         if offsets[start_index] is None or offsets[end_index] is None:
#           continue

#         # Skip answers with a length that is either < 0 or > max_answer_length.
#         if (
#           end_index < start_index
#           or end_index - start_index + 1 > self.max_answer_length
#         ):
#           continue

#         answers.append(
#           {
#             "text": context[offsets[start_index][0] : offsets[end_index][1]],
#             "logit_score": start_logit[start_index] + end_logit[end_index],
#           }
#         )

#     best_answer = max(answers, key=lambda x: x["logit_score"])
#     return best_answer["text"].strip()

In [ ]:
test_question = """Làm thế nào để nộp chứng chỉ ngoại ngữ?"""

In [ ]:
# inputs = tokenizer(test_question, test_context,
#         max_length=max_length,
#         truncation="only_second",
#         stride=stride,
#         # return_overflowing_tokens=True,
#         # return_offsets_mapping=True,
#         padding="max_length",
#         return_tensors="pt")

In [ ]:
# inputs

In [ ]:
# inputs = {key: value.to("cuda") for key, value in inputs.items()}

In [ ]:
# inputs

In [ ]:
# with torch.no_grad():
#   outputs = model(**inputs)

In [ ]:
# outputs

In [ ]:
# print(get_prediction(test_question, all_contexts))

In [ ]:
new_column = [test_question] * len(all_contexts)
eval_long_set = all_contexts.add_column("question", new_column)
# eval_long_set = contexts.map(lambda context: {"context": context, "question": question})
print(eval_long_set[1])


{'context': '6. Chuẩn kỹ năng ứng dụng Công nghệ thông tin: Từ khóa 2021, sinh viên phải đạt chuẩn kỹ năng ứng dụng Công nghệ thông tin cơ bản và đạt chứng chỉ MOS (Microsoft Office Specialist) ở 02 bài thi là Excel và PowerPoint. 7. Các điều kiện ràng buộc: Sinh viên phải đạt chuẩn kỹ năng ứng dụng Công nghệ thông tin cơ bản sau năm thứ nhất để có thể được xếp trình độ năm thứ hai. Sinh viên phải đạt chứng chỉ MOS để được đăng ký học phần Khoá luận tốt nghiệp. 8. Cách thức đạt chuẩn: Nhà trường tổ chức các kỳ kiểm tra kỹ năng ứng dụng Công nghệ thông tin nội bộ, không cấp chứng chỉ, miễn phí một lần cho toàn bộ các sinh viên chính quy trong năm thứ nhất. Sinh viên có thể nộp các chứng chỉ Ứng dụng Công nghệ thông tin cơ bản hợp lệ hoặc chứng chỉ quốc tế IC3 để hoàn tất chuẩn này. Trường phối hợp với các đơn vị có trách nhiệm tổ chức các kỳ thi lấy chứng chỉ MOS cho sinh viên. Mỗi sinh viên chính quy được miễn phí một lần thi. Sinh viên có thể nộp các chứng chỉ MOS tương ứng để hoàn tấ

In [ ]:
max_length = 1024
stride = 256

In [ ]:
def preprocess_dataset(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_map = inputs.pop("overflow_to_sample_mapping")
    example_ids = []

    for i in range(len(inputs["input_ids"])):
        sample_idx = sample_map[i]
        example_ids.append(examples["id"][sample_idx])

        sequence_ids = inputs.sequence_ids(i)
        offset = inputs["offset_mapping"][i]
        inputs["offset_mapping"][i] = [
            o if sequence_ids[k] == 1 else None for k, o in enumerate(offset)
        ]

    inputs["example_id"] = example_ids
    inputs["overflow_to_sample_mapping"] = sample_map
    return inputs

In [ ]:
eval_set = eval_long_set.map(
preprocess_dataset,
batched=True,
remove_columns=eval_long_set.column_names,
load_from_cache_file=False
)

Map:   0%|          | 0/42 [00:00<?, ? examples/s]

In [ ]:
eval_set

Dataset({
    features: ['input_ids', 'attention_mask', 'offset_mapping', 'example_id', 'overflow_to_sample_mapping'],
    num_rows: 64
})

In [ ]:
overflow_to_sample_mapping = eval_set["overflow_to_sample_mapping"]

In [ ]:
eval_set = eval_set.remove_columns(["overflow_to_sample_mapping"])

In [ ]:
length = eval_set.num_rows
print(length)

64


In [ ]:
eval_set_for_model = eval_set.remove_columns(["example_id", "offset_mapping"])
eval_set_for_model.set_format("torch")

In [ ]:
torch.cuda.empty_cache()

In [ ]:
limit = 50

In [ ]:
import numpy as np

In [ ]:
# batch = {k: eval_set_for_model[k].to("cuda") for k in eval_set_for_model.column_names}
i = 0

start_logits = []
end_logits = []
while i <= length:
  start = i
  i += limit
  if(i<length):
    end = i
  else:
    end = length
  print("Batched at %s index to %s index, all: %s" % (start, end-1, length))

  batch = {k: eval_set_for_model[k][start:end].to("cuda") for k in eval_set_for_model.column_names}

  with torch.no_grad():
    outputs = model(**batch)
    start_logits.append(outputs["start_logits"])
    end_logits.append(outputs["end_logits"])

  print(len(start_logits), len(end_logits))

start_logits = torch.cat(start_logits, dim=0)
end_logits = torch.cat(end_logits, dim=0)

start_logits = start_logits.detach().cpu().numpy()
end_logits = end_logits.detach().cpu().numpy()

Batched at 0 index to 49 index, all: 64
1 1
Batched at 50 index to 63 index, all: 64
2 2


In [ ]:
len(end_logits)

64

In [ ]:
import collections

In [ ]:
example_to_features = collections.defaultdict(list)
for idx, feature in enumerate(eval_set):
  example_to_features[feature["example_id"]].append(idx)

In [ ]:
len(all_contexts)

42

In [ ]:
n_best = 10
max_answer_length = 1024

In [ ]:
eval_set

Dataset({
    features: ['input_ids', 'attention_mask', 'offset_mapping', 'example_id'],
    num_rows: 64
})

In [ ]:
answers=[]
for index in range(len(start_logits)):
  start_logit = start_logits[index]
  end_logit = end_logits[index]
  offsets = eval_set["offset_mapping"][index]
  context = all_contexts[overflow_to_sample_mapping[index]]['context']
  start_indexes = np.argsort(start_logit)[-1 : - n_best - 1 : -1].tolist()
  end_indexes = np.argsort(end_logit)[-1 : - n_best - 1 : -1].tolist()

  for start_index in start_indexes:
   for end_index in end_indexes:
    # Skip answers that are not fully in the context
      if offsets[start_index] is None or offsets[end_index] is None:
        continue

      # Skip answers with a length that is either < 0 or > max_answer_length.
      if (
        end_index < start_index
       or end_index - start_index + 1 > max_answer_length
      ):
        continue

      answers.append(
        {
          "text": context[offsets[start_index][0] : offsets[end_index][1]],
         "logit_score": start_logit[start_index] + end_logit[end_index],
       }
     )
      print("text :%s at point: %s"% (context[offsets[start_index][0] : offsets[end_index][1]], start_logit[start_index] + end_logit[end_index]))

best_answer = max(answers, key=lambda x: x["logit_score"])
print(best_answer)

text :a) Tích lũy đủ học phần, số tín chỉ của chương trình đào tạo, đạt chuẩn đầu ra của chương trình đào tạo; at point: 10.09659
text :a) Tích lũy đủ học phần, số tín chỉ của chương trình đào tạo, đạt chuẩn đầu ra của chương trình đào tạo; b) Hoàn thành chương trình Giáo dục Quốc phòng - An ninh và chương trình Giáo dục Thể chất; c) Hoàn thành chương trình rèn luyện sinh viên theo quy định của nhà trường; d) Đạt chuẩn ngoại ngữ tốt nghiệp và các chuẩn khác của nhà trường; e) Điểm trung bình tích lũy của toàn khóa học đạt từ trung bình trở lên; f) Tại thời điểm xét tốt nghiệp không bị truy cứu trách nhiệm hình sự hoặc không đang trong thời gian bị kỷ luật ở mức đình chỉ học tập. at point: 9.175055
text :a) Tích lũy đủ học phần, số tín chỉ của chương trình đào tạo, đạt chuẩn đầu ra của chương trình đào tạo; b) Hoàn thành chương trình Giáo dục Quốc phòng - An ninh và chương trình Giáo dục Thể chất; c) Hoàn thành chương trình rèn luyện sinh viên theo quy định của nhà trường; d) Đạt chuẩn 

In [ ]:
print(test_question)
print()
print(best_answer)

Làm thế nào để nộp chứng chỉ ngoại ngữ?

{'text': 'Sinh viên có thể nộp các chứng chỉ Ứng dụng Công nghệ thông tin cơ bản hợp lệ hoặc chứng chỉ quốc tế IC3 để hoàn tất chuẩn này.', 'logit_score': 16.087187}


(0, 0)

In this case we’ve only added a few samples, so it appears the contexts in the validation dataset are a bit shorter.

In [ ]:
small_eval_set = raw_datasets["validation"].select(range(100))
trained_checkpoint = "distilbert-base-cased-distilled-squad"

eval_set = small_eval_set.map(
    preprocess_validation_dataset,
    batched=True,
    remove_columns=raw_datasets["validation"].column_names,
)

In [ ]:
import torch
from transformers import AutoModelForQuestionAnswering

eval_set_for_model = eval_set.remove_columns(["example_id", "offset_mapping"])
eval_set_for_model.set_format("torch")

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
batch = {k: eval_set_for_model[k].to(device) for k in eval_set_for_model.column_names}
trained_model = AutoModelForQuestionAnswering.from_pretrained(MODEL).to(
    device
)

with torch.no_grad():
    outputs = trained_model(**batch)

In [ ]:
start_logits = outputs.start_logits.cpu().numpy()
end_logits = outputs.end_logits.cpu().numpy()

In [ ]:
import collections

example_to_features = collections.defaultdict(list)
for idx, feature in enumerate(eval_set):
    example_to_features[feature["example_id"]].append(idx)